# Step 1 — Preprocess exported GEE CSVs

**Input:** multiple CSV shards exported from Google Earth Engine (Sentinel-1 10-day VV/VH composites sampled on polygons).  
**Output:** one merged training table in `data/prepared/` with:

- `POINT_ID`, `stratum`, `level_1`, `level_2`
- feature columns like `VV_YYYYMMDD`, `VH_YYYYMMDD` (optionally `R_YYYYMMDD`)

Run this after finishing the GEE export tasks.

In [ ]:
import os
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """
    Find repository root by looking for README.md and notebooks/ folder.
    Works in local Jupyter + Colab after git clone.
    """
    p = (start or Path.cwd()).resolve()
    for _ in range(8):
        if (p / "README.md").exists() and (p / "notebooks").exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return (start or Path.cwd()).resolve()

REPO_ROOT = find_repo_root()

DATA_RAW = REPO_ROOT / "data" / "raw"
DATA_PREP = REPO_ROOT / "data" / "prepared"

DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PREP.mkdir(parents=True, exist_ok=True)

print("REPO_ROOT:", REPO_ROOT)
print("DATA_RAW:", DATA_RAW)
print("DATA_PREP:", DATA_PREP)

In [ ]:
# --- 1_Legend_polygons_COPERNICUS_reorganise_trimmed.py ---

import pandas as pd
import numpy as np
from pathlib import Path
import glob, os, re

DATA_PATH = DATA_RAW
OUT_PATH = DATA_PREP

# Helper: list what we see
all_files = sorted(list(DATA_PATH.glob("*.csv")) + list(DATA_PATH.glob("*.CSV")))
print(f"Found {len(all_files)} CSVs in {DATA_PATH}")
for f in all_files[:5]:
    print(" •", f.name)
if not all_files:
    raise FileNotFoundError(f"No CSV files found in {DATA_PATH}")

# 1) Build the metadata table (POINT_ID, LC1, LU1, stratum) from shards
usecols = ["POINT_ID", "LC1", "LU1", "stratum"]
meta_parts = []
for f in all_files:
    # Read header first to keep it fast and robust
    hdr = pd.read_csv(f, nrows=0).columns.tolist()
    cols_present = [c for c in usecols if c in hdr]
    df = pd.read_csv(f, usecols=cols_present)
    meta_parts.append(df)

lucas_polygons = (pd.concat(meta_parts, ignore_index=True)
                    .drop_duplicates(subset=["POINT_ID"])
                    .reset_index(drop=True))

# drop non-relevant LC1 classes (same as original)
drop_set = ['A30','H11','H12','H21','H22','H23','G21','G30','G11']
lucas_polygons = lucas_polygons[~lucas_polygons.LC1.isin(drop_set)].copy()

# LU1s recode (same as original)
lucas_polygons['LU1s'] = lucas_polygons['LU1'].astype(str)
lucas_polygons['LU1s'] = lucas_polygons['LU1s'].replace(['U111','U112','U113'],'U1')
lucas_polygons['LU1s'] = lucas_polygons['LU1s'].replace(
    ['U120','U420','U415','U361','U362','U350','U370','U330','U150','U411','U412','U140',
     'U312','U341','U314','U414','U210','U319','U318','U222','U321','U322','U413','U311',
     'U317','U342','U313','U224','U226','U223','U316','U315','U225','U221','U227'], 'U0'
)

# Legend mapping to level_2 (exactly as before)
def repl(s, pat, val): return s.str.replace(pat, val, regex=True)
lucas_polygons['LC1_LU1'] = lucas_polygons['LC1'].astype(str) + lucas_polygons['LU1s'].astype(str)
lucas_polygons['level_2'] = lucas_polygons['LC1_LU1']

for pat, val in [
    (r'A[12][0123]U[01]','100'),
    (r'B11U[01]','211'),(r'B12U[01]','212'),(r'B13U[01]','213'),
    (r'B14U[01]','214'),(r'B15U[01]','215'),(r'B16U[01]','216'),
    (r'B17U[01]','217'),(r'B18U[01]','218'),(r'B19U[01]','219'),
    (r'B21U[01]','221'),(r'B22U[01]','222'),(r'B23U[01]','223'),
    (r'B3[4-7]U[01]','230'),(r'B31U[01]','231'),(r'B32U[01]','232'),(r'B33U[01]','233'),
    (r'B4[1-5]U[01]','240'),(r'B5[1-4]U[01]','250'),(r'B55U[01]','500'),
    (r'B7[1-7]U[01]','300'),(r'B8[1-4]U[01]','300'),
    (r'C[123][0123]U[01]','300'),(r'D[123][0123]U[01]','300'),
    (r'E[123][0123]U[1]','500'),
    (r'E[1][0123]U[0]','500'),(r'E[2][0123]U[0]','500'),(r'E[3][0123]U[0]','500'),
    (r'F40U1','290'),(r'F[123][0]U[01]','600'),(r'F40U0','600'),
]:
    lucas_polygons['level_2'] = repl(lucas_polygons['level_2'], pat, val)

# level_1 from level_2
lucas_polygons['level_1'] = lucas_polygons['level_2']
lucas_polygons['level_1'] = lucas_polygons['level_1'].str.replace(r'2[1-6][0-9]', '200', regex=True)
lucas_polygons['level_1'] = lucas_polygons['level_1'].str.replace(r'2[7][0-9]', '300', regex=True)
lucas_polygons['level_1'] = lucas_polygons['level_1'].str.replace(r'2[8][0-9]', '300', regex=True)
lucas_polygons['level_1'] = lucas_polygons['level_1'].str.replace(r'290', '200', regex=True)

lucas_polygons = lucas_polygons[['POINT_ID','stratum','LC1','LU1','level_1','level_2']].copy()
for col in ['level_1','level_2']:
    lucas_polygons[col] = pd.to_numeric(lucas_polygons[col], errors='ignore')

legend_csv = OUT_PATH / "LUCAS_2018_Copernicus_attributes_cropmap_level1-2_FROM_EXPORTS.csv"
lucas_polygons.to_csv(legend_csv, index=False)
print("Saved:", legend_csv)


Found 8 CSVs in /content/drive/MyDrive/EU_subset
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1a_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1b_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1c_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1d_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE2_ratio-db.csv
Saved: /content/drive/MyDrive/EU_subset/prepared/LUCAS_2018_Copernicus_attributes_cropmap_level1-2_FROM_EXPORTS.csv


/tmp/ipython-input-2452112793.py:86: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  lucas_polygons[col] = pd.to_numeric(lucas_polygons[col], errors='ignore')


In [ ]:
print(f"[Script1] Total CSVs found: {len(all_files)}")
if all_files:
    print("[Script1] First 2 files:")
    for f in all_files[:2]:
        print("  -", f.name)
    # Peek at the very first CSV
    _tmp = pd.read_csv(all_files[0], nrows=5)
    print("\n[Script1] HEAD of FIRST INPUT CSV:")
    print(_tmp.head().to_string(index=False))


[Script1] Total CSVs found: 8
[Script1] First 2 files:
  - S1_point_all_10d_10m_20180101-20180731_EU_NE1a_ratio-db.csv
  - S1_point_all_10d_10m_20180101-20180731_EU_NE1b_ratio-db.csv

[Script1] HEAD of FIRST INPUT CSV:
          system:index LC1  LU1  POINT_ID  VH_20180101  VH_20180111  VH_20180121  VH_20180131  VH_20180210  VH_20180220  VH_20180302  VH_20180312  VH_20180322  VH_20180401  VH_20180411  VH_20180421  VH_20180501  VH_20180511  VH_20180521  VH_20180531  VH_20180610  VH_20180620  VH_20180630  VH_20180710  VH_20180720  VV_20180101  VV_20180111  VV_20180121  VV_20180131  VV_20180210  VV_20180220  VV_20180302  VV_20180312  VV_20180322  VV_20180401  VV_20180411  VV_20180421  VV_20180501  VV_20180511  VV_20180521  VV_20180531  VV_20180610  VV_20180620  VV_20180630  VV_20180710  VV_20180720  stratum                                   .geo
00010000000000001692_0 B11 U111  47242864   -17.729420   -20.325294   -19.684908   -20.850082   -20.764990   -23.271540   -20.451742   -20.227938

In [ ]:
print("\n[Script1] HEAD of legend table BEFORE SAVE:")
print(lucas_polygons.head().to_string(index=False))



[Script1] HEAD of legend table BEFORE SAVE:
 POINT_ID  stratum LC1  LU1  level_1  level_2
 47242864        1 B11 U111      200      211
 47322804        1 B11 U111      200      211
 47602810        1 B11 U111      200      211
 47622814        1 B11 U111      200      211
 47642818        1 B11 U111      200      211


In [ ]:
# --- 2_Polygons_extract_GEE_reorganize_trimmed_S1.py ---

import pandas as pd
import numpy as np
from pathlib import Path
import glob, os, re

# from google.colab import drive
# drive.mount('/content/drive')


DATA_PATH = DATA_RAW
OUT_PATH = DATA_PREP

# Show what we see
all_files = sorted(list(DATA_PATH.glob("*.csv")) + list(DATA_PATH.glob("*.CSV")))
print(f"Found {len(all_files)} CSVs in {DATA_PATH}")
for f in all_files[:5]:
    print(" •", f.name)
if not all_files:
    raise FileNotFoundError(f"No CSV files found in {DATA_PATH}")

# 1) Legend table from script 1
legend_csv = OUT_PATH / "LUCAS_2018_Copernicus_attributes_cropmap_level1-2_FROM_EXPORTS.csv"
lucas_polygons = pd.read_csv(legend_csv, dtype={'level_1':'Int64','level_2':'Int64'})

# 2) Load S1 shards
parts = []
for f in all_files:
    df = pd.read_csv(f)
    # Drop geometry column only if present
    if '.geo' in df.columns:
        df = df.drop(columns=['.geo'])
    parts.append(df)

pd_lucas = pd.concat(parts, ignore_index=True)
# ==== DEBUG A: does pd_lucas already have 'stratum'? ====
print("[DBG] pd_lucas has 'stratum'? ->", 'stratum' in pd_lucas.columns)
print("[DBG] First 5 columns of pd_lucas:", pd_lucas.columns[:5].tolist())

# 3) Merge with legend (only POINT_IDs that exist in legend)
merged = pd.merge(
    pd_lucas,
    lucas_polygons[['POINT_ID','stratum','level_2','level_1']],
    on='POINT_ID',
    how='inner'
)
# ==== DEBUG B: what did the merge produce? ====
print("[DBG] Columns after merge (first 20):", merged.columns[:20].tolist())
s_cols = [c for c in merged.columns if 'stratum' in c]
print("[DBG] Any 'stratum'-like columns:", s_cols)
# ==== FIX: make sure we have a clean 'stratum' column ====
if 'stratum' not in merged.columns:
    if 'stratum_y' in merged.columns:
        merged['stratum'] = merged['stratum_y']
    elif 'stratum_x' in merged.columns:
        merged['stratum'] = merged['stratum_x']

# (Optional) drop the extras to avoid confusion
for c in ['stratum_x', 'stratum_y']:
    if c in merged.columns:
        merged = merged.drop(columns=[c])

# 4) Keep only complete rows; then optionally keep stratum 1
merged = merged.dropna(subset=['level_1','level_2','stratum'])
merged = merged[merged['stratum'] == 1].copy()

# 5) Stable feature order; supports VV/VH with optional R_
band_cols = sorted([c for c in merged.columns if re.match(r'^(VV_|VH_|R_)\d{8}$', c)])
id_cols = [c for c in ['POINT_ID','stratum'] if c in merged.columns]
label_cols = [c for c in ['level_1','level_2'] if c in merged.columns]
merged = merged[id_cols + label_cols + band_cols]
print("[DBG] HEAD of MERGED (with clean 'stratum'):")
print(merged[['POINT_ID','stratum','level_1','level_2']].head().to_string(index=False))

# 6) Save with a truthful name (Jan–Jul, Stratum1, VV-VH, add -RATIO if added)
out_csv = OUT_PATH / "S1_point_all_10d_10m_20180101-20180731_Stratum1_VV-VH.csv"
merged.to_csv(out_csv, index=False)
print("Saved:", out_csv, "Shape:", merged.shape)
print("Top level_2 classes:\n", merged[['POINT_ID','level_2']].drop_duplicates()['level_2'].value_counts().head(15))


Found 8 CSVs in /content/drive/MyDrive/EU_subset
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1a_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1b_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1c_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE1d_ratio-db.csv
 • S1_point_all_10d_10m_20180101-20180731_EU_NE2_ratio-db.csv
[DBG] pd_lucas has 'stratum'? -> True
[DBG] First 5 columns of pd_lucas: ['system:index', 'LC1', 'LU1', 'POINT_ID', 'VH_20180101']
[DBG] Columns after merge (first 20): ['system:index', 'LC1', 'LU1', 'POINT_ID', 'VH_20180101', 'VH_20180111', 'VH_20180121', 'VH_20180131', 'VH_20180210', 'VH_20180220', 'VH_20180302', 'VH_20180312', 'VH_20180322', 'VH_20180401', 'VH_20180411', 'VH_20180421', 'VH_20180501', 'VH_20180511', 'VH_20180521', 'VH_20180531']
[DBG] Any 'stratum'-like columns: ['stratum_x', 'stratum_y']
[DBG] HEAD of MERGED (with clean 'stratum'):
 POINT_ID  stratum  level_1  level_2
 47242864        1      200      211
 4724

       POINT_ID  stratum  LC1   LU1  level_1  level_2
0      47242864        1  B11  U111      200      211
1      47322804        1  B11  U111      200      211
2      47602810        1  B11  U111      200      211
3      47622814        1  B11  U111      200      211
4      47642818        1  B11  U111      200      211
...         ...      ...  ...   ...      ...      ...
30929  36503184        1  E20  U111      500      500
30930  36643156        1  E20  U111      500      500
30931  36703150        1  E20  U111      500      500
30932  36503170        1  F40  U111      200      290
30933  36883144        1  F40  U112      200      290

[30934 rows x 6 columns]


Loaded dataset with shape (1743815, 46)


Classes in level_1: [200, 500, 300, 600, 100]
Classes in level_2: [211, 212, 213, 214, 215, 216, 218, 219, 221, 222, 223, 231, 232, 233, 230, 240, 250, 290]
   POINT_ID  stratum  level_1  level_2  VH_20180101  VH_20180111  VH_20180121  \
0  47242864        1      200      211   -17.729420   -20.325294   -19.684908   
1  47242864        1      200      211   -17.629759   -20.395664   -19.362911   
2  47322804        1      200      211   -16.761300   -16.439291   -19.003990   
3  47322804        1      200      211   -16.949911   -17.447950   -18.359556   
4  47322804        1      200      211   -16.443756   -16.525919   -18.132175   

   VH_20180131  VH_20180210  VH_20180220  ...  VV_20180421  VV_20180501  \
0   -20.850082   -20.764990   -23.271540  ...   -15.297538   -14.691077   
1   -20.440153   -21.169271   -23.260570  ...   -13.002155   -14.203595   
2   -20.409580   -22.091795   -21.210240  ...   -16.568722   -17.411484   
3   -20.564657   -23.349674   -21.620611  ...   -16.0513

Data after filtering to crop classes: (604610, 47)


[1]


Selected features matrix X shape: (604610, 42)
Selected target vector y shape: (604610,)
Class distribution in y:
Classif
211    195821
213     89215
216     69089
232     53663
214     31153
290     24290
215     24001
250     23366
222     22901
240     18351
218     16124
221     14620
230      6360
212      4711
223      3780
219      3446
231      1975
233      1744
Name: count, dtype: int64


Baseline Random Forest trained.


NameError: name 'baseline_rf' is not defined

NameError: name 'Pipeline' is not defined